In [ ]:

train_prefix="multiclass_indep_classifier_"

train_folder="./train-val-independent/train/"
valid_folder="./train-val-independent/validation/"
train_csv = "./csv_resnet_indep/trainData.csv"
valid_csv = "./csv_resnet_indep/valiData.csv"

b_selfattention=True

batch_size = 4
learning_rate = 1e-3
epochs = 500

start_model_path = None #"c:/PROJECTS/szunyog/b_mosquito/b_mosquito_simple_classifier-1_checkpoints/best_loss_99.pth"


In [ ]:
import dataPreprocess
import model_modified
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import time
import os



In [ ]:
# create csv-s for classification folders
# Fname,Genera,Species

import os
import csv

# create folders for csvs
os.makedirs(os.path.dirname(train_csv), exist_ok=True)
os.makedirs(os.path.dirname(valid_csv), exist_ok=True)


def generate_csv(folder_path, output_csv):
    """
    generate csv from paths.
    """
    data = []
    label_list=set()

    # for all subfolders and files
    for species in os.listdir(folder_path):
        label_list.add(species)
        
        species_path = os.path.join(folder_path, species)
        if os.path.isdir(species_path):
            for fname in os.listdir(species_path):
                # add files with their data
                if os.path.isfile(os.path.join(species_path, fname)):
                    data.append({
                        "Fname": os.path.join(species_path, fname).replace("\\", "/"),
                        "Genera": "",
                        "Species": species
                    })

    # write out CSV
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["Fname", "Genera", "Species"])
        writer.writeheader()
        writer.writerows(data)

    return {key: idx for idx, key in enumerate(label_list)}

# Train CSV
label_list=generate_csv(train_folder, train_csv)
print(f"Train CSV is created: {train_csv}")

# Validation CSV
generate_csv(valid_folder, valid_csv)
print(f"Validation CSV is created: {valid_csv}")

print(label_list)


In [ ]:
def train(model, device, train_loader, valid_loader, epochs, lf, optimizer):
    sched = optim.lr_scheduler.CosineAnnealingLR(optimizer, 200, 0)
    best_valid_acc = 0
    best_model_report = ''

    output_file = open(train_prefix+'output_big.txt', 'w+')
    output_file.write('start...')
    output_file.write('\n')
    output_file.close()
    print('start...')
    # training procedure
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        model.train()
        loss_val = 0
        true_running = 0
        total_running = 0
        for i, data in enumerate(train_loader):
            #print(data)
            x, gt = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.long)
            optimizer.zero_grad()
            predicted = model(x)
            loss = lf(predicted, gt)

            result, predicted_class = torch.max(predicted, 1)
            true_running += (predicted_class == gt).sum()
            total_running += predicted_class.shape[0]

            loss.backward()
            optimizer.step()

            loss_val += loss.item()

        train_loss = loss_val / len(train_loader)
        accuracy = torch.true_divide(true_running, total_running)
        print(f'Epoch - {epoch} Train - Loss : {train_loss} Accuracy : {accuracy}')
        output_file = open(train_prefix+'output_big.txt', 'a')
        output_file.write(f'Epoch {epoch}/{epochs} - Train')
        output_file.write(f'loss: {train_loss}')
        output_file.write('\n')
        output_file.write(f'accuracy: {accuracy}')
        output_file.write('\n')
        output_file.close()

        sched.step()
        model.eval()

        # validating procedure
        valid_loss_val = 0
        valid_true_running = 0
        valid_total_running = 0
        y_pred = np.array([])
        y_test = np.array([])
        for i, data in enumerate(valid_loader):
            x, gt = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.long)
            predicted = model(x)
            loss = lf(predicted, gt)

            result, predicted_class = torch.max(predicted, 1)
            valid_true_running += (predicted_class == gt).sum()
            valid_total_running += predicted_class.shape[0]

            valid_loss_val += loss.item()

            y_pred = np.append(y_pred, predicted_class.cpu().detach().numpy())
            y_test = np.append(y_test, gt.cpu().detach().numpy())

        # calculating measurements
        valid_loss = valid_loss_val / len(train_loader)
        accuracy = torch.true_divide(valid_true_running, valid_total_running)
        print(f'Epoch - {epoch} Validation - Loss : {valid_loss} Accuracy : {accuracy}')

        # accuracy and loss
        output_file = open(train_prefix+'output_big.txt', 'a')
        output_file.write(f'Epoch {epoch}/{epochs} - Validation')
        output_file.write(f'loss: {valid_loss}')
        output_file.write('\n')
        output_file.write(f'accuracy: {accuracy}')
        output_file.write('\n')

        # precision, recall, f1-score
        output_file.write('\nClassification Report\n')
        output_file.write(classification_report(y_test, y_pred, zero_division=0))
        output_file.write('\n')

        # confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        output_file.write('\nConfusion Matrix\n')
        output_file.write(str(conf_matrix))
        output_file.write('\n')

        # time usage for each epoch
        end_time = time.time()
        usage_time = end_time - start_time
        output_file.write(f'Time usage: {usage_time} secs')
        output_file.write('\n')
        output_file.write('\n')

        output_file.close()

        # save best model and its performance report, can be used for futher training
        
        # Save best loss model
        if epoch == 1 or valid_loss < best_loss_val:
            best_loss_val = valid_loss
            torch.save(model.state_dict(), f'./{train_prefix}checkpoints/best_loss_{epoch}.pth')

        # Save best accuracy model
        if accuracy > best_valid_acc:
            best_valid_acc = accuracy
            best_model_report = classification_report(y_test, y_pred, zero_division=0)
            torch.save(model.state_dict(), f'./{train_prefix}checkpoints/big_{epoch}.pth')

        # Save the last model
        torch.save(model.state_dict(), f'./{train_prefix}checkpoints/last_model.pth')

        # report the best training model
        if epoch == epochs:
            output_file = open(train_prefix+'output.txt', 'a')
            output_file.write(f'End Training Overall Report')
            output_file.write('\n')
            output_file.write(f'Best Validation Accuracy: {best_valid_acc}')
            output_file.write('\n')
            output_file.write(f'Classification Report: {best_model_report}')
            output_file.write('\n')
            output_file.write(f'The best model is saved under resnet_attention.pth')
            output_file.close()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
if train_prefix+"checkpoints" not in os.listdir("./"):
    os.mkdir(f"./{train_prefix}checkpoints")
else:
    print("Checkpoints folder is exist.")


loss_function = nn.CrossEntropyLoss()


In [ ]:

cls_num=len(label_list)

train_data = dataPreprocess.ListDataset(train_csv, label_list, "train")
vali_data = dataPreprocess.ListDataset(valid_csv, label_list, "validation")

print('loading training data')
train_loader = DataLoader(train_data, batch_size, shuffle=True)
print('loading validation data')
vali_loader = DataLoader(vali_data, batch_size, shuffle=False)


In [ ]:
# load model trained allready fora a binary classification

if start_model_path is not None: 
    resnet_model = model_modified.resnet18_attention(1, 2, b_selfattention=b_selfattention)

    resnet_model.load_state_dict(torch.load(start_model_path))
    # redefine classification layer
    resnet_model.decoder.decoder[3] = nn.Linear(in_features=128, out_features=cls_num, bias=True)

    # initialize weights for the last layer
    nn.init.xavier_uniform_(resnet_model.decoder.decoder[3].weight)
    nn.init.zeros_(resnet_model.decoder.decoder[3].bias)
else:
    resnet_model = model_modified.resnet18_attention(1, cls_num, b_selfattention=b_selfattention)


In [ ]:

resnet_model = resnet_model.to(device)

resnet_model


In [ ]:
resnet_model = resnet_model.to(device)
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)

train(resnet_model, device, train_loader, vali_loader, epochs, loss_function, optimizer)
